In [17]:
import os
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from lmfit import models
from pathlib import Path
from astropy.io import fits
import scipy.constants as sc
import astropy.constants as ac

In [18]:
def pixels_to_wavelenghts(pixel, slope, intercept):
    '''
    This function converts pixel values to wavelengths using a linear model to be determined by a fit.
    '''
    return slope * pixel + intercept

def import_files(path, filename):
    '''
    Import files in the syntax that I used in the notebook. That is, choose the path that contains
    all the fit files, then for the filename choose something like bias* to obtain all bias files
    as a dictionary that can be accessed by filename 
    '''
    files =  [ p for p in Path(path).glob(filename) ]
    file_dict = dict()
    for file in files:
        file_dict[os.path.basename(file)] = fits.getdata(file) # key of the dict is the filename.

    headers_dict = dict()
    for x in file_dict.keys():
        headers_dict[x] = fits.getheader(path + x)

    return file_dict, headers_dict 

def get_peak_wavelength(wavelengths, irradiances):
    index = list(irradiances).index(max(irradiances))
    return  wavelengths[index]

def get_effective_temperature(wavelength_peak):
    return ( 2.897771955 * 10**-3 ) / (wavelength_peak * 10**-9)

def stack(values):
    return  np.median( [ x for x in values ], axis=0 )

def planck_scaled(temperature, wavelength):
    return (S := ( ( ( ac.R_sun / ac.au ) ** 2 )  *  ( 2 * sc.Planck * sc.speed_of_light ** 2 ) /
        ( (wavelength**5) * ( np.exp( ( sc.Planck * sc.speed_of_light) / ( wavelength * sc.k * temperature ) ) - 1 ) ) ) )


Import fraunhofer lines

In [19]:
df_fl = pd.read_csv('../data/fraunhofer_lines.csv')
df_fl.keys()

Index(['name', 'wavelength', 'pixel', 'intensity'], dtype='object')

In [20]:
flat_dict, flat_headers = import_files(
    "C:/Users/bvptr/academia/physics/year2/natuurkunde_en_sterrenkunde_practicum2/solar_physics_non_git/solar_physics_data/NP2 zonnefysica/20211112 - LISA daglicht spectra/" ,
    "flat_alt*"
)

light_dict, light_headers = import_files(
    "C:/Users/bvptr/academia/physics/year2/natuurkunde_en_sterrenkunde_practicum2/solar_physics_non_git/solar_physics_data/20201109 sky zenith/" ,
    "skyspectrum*")

dark_dict, dark_headers = import_files(
    "C:/Users/bvptr/academia/physics/year2/natuurkunde_en_sterrenkunde_practicum2/solar_physics_non_git/solar_physics_data/20201109 sky zenith/" ,
    "Dark*"
)

bias_dict, bias_headers = import_files(
    "C:/Users/bvptr/academia/physics/year2/natuurkunde_en_sterrenkunde_practicum2/solar_physics_non_git/solar_physics_data/NP2 zonnefysica/SunsetLISA/" ,
    "bias*"
)

print(
    light_headers['skyspectrum zenit-0001.fit']['IMAGETYP'],
    flat_headers['flat_alt-0001.fit']['IMAGETYP'],
    dark_headers['Dark-0001.fit']['IMAGETYP'],
    bias_headers['bias-001.fit']['IMAGETYP'],
)

Light Frame Light Frame Dark Frame Bias Frame


### Calibration

In [21]:
model = models.LinearModel()
fit_pixel_wavelength = model.fit(x=df_fl['pixel'], data=df_fl['wavelength'])
#print(fit_pixel_wavelength.params['slope'])
#print(fit_pixel_wavelength.params['intercept'])
#fit_pixel_wavelength.plot(xlabel='pixel', ylabel='Wavelength (nm)')

### Stacking

In [22]:
light_stack = stack( light_dict.values() )
bias_master = stack( bias_dict.values() )

In [23]:
flat_master = stack( [flat - bias_master for flat in flat_dict.values() ] )

In [24]:
flat_master = flat_master / np.median(flat_master) # scale flat_master

In [12]:
calibrated = [ (light - bias_master) / flat_master for light in light_stack ]

In [25]:
spectrum = np.median(calibrated[:], axis=0)

MemoryError: Unable to allocate 11.2 GiB for an array with shape (1039, 1039, 1391) and data type float64

In [ ]:
light_sum = np.sum( light_stack, axis=0)

In [ ]:
calibrated_sum = np.sum(calibrated, axis=0)

Convert to wavelenghts

In [14]:
wavelengths = pixels_to_wavelenghts(range(len(spectrum)), fit_pixel_wavelength.values['slope'], fit_pixel_wavelength.values['intercept'])

NameError: name 'spectrum' is not defined

### crude method to determine effective temperature.

In [ ]:
T_accepted = 5772
print(wavelength_peak := get_peak_wavelength(wavelengths, spectrum)) # 481.6998282285298
print(T_eff := get_effective_temperature(wavelength_peak)) # 6015.72137913495
print("%f%% error from the accepted value." % float( ( T_eff / T_accepted ) * 100 - 100) ) #4.22% error

In [ ]:
plt.xlabel('wavelength (nm)')
plt.ylabel('intensity (counts)')
plt.plot(wavelengths, spectrum)

In [ ]:
df = pd.DataFrame({'spectral irradiance' : spectrum, 'wavelength' : wavelengths})
df.to_csv("./data/to_fit.csv")